In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone
from urllib.parse import urlparse
from dateutil import parser
import unicodedata
import numpy as np
import os

# === CONFIGURACIÓN GENERAL ===

# 1) Términos ligados a intereses de clientes DAP
terminos_busqueda = [
    "industria alimentaria",
    "cemento",
    "gas",
    "impuesto", 
    "iva",
    "casinos",
    "movilidad",
    "seguridad",
]

# 2) Búsqueda general para agenda nacional
termino_agenda_nacional = ["Sheinbaum","aranceles","trump"]

# Modo de fecha
modo = "rango"  # Opciones: "exacto" o "rango"
debug = True    # True para imprimir detalles por noticia
dias_rango = 2  # Solo si modo == "rango"
fecha_objetivo = datetime(2025, 11, 2).date()  # Solo si modo == "exacto"

# Lista de medios (de momento vacía: tú la llenas con tus sitios)
medios = ["eleconomista.com.mx","imagenradio.com.mx","elfinanciero.com.mx","forbes.com.mx","merca20.com",
    "eluniversal.com.mx","heraldodemexico.com.mx/nacional","nexos.com.mx","thelogisticsworld.com","t21.com.mx",
    "articulo19.org","animalpolitico.com","sinembargo.mx","codigomagenta.com.mx","latinus.us","expansion.mx",
    "nmas.com.mx/nacional","radioformula.com.mx","wradio.com.mx","unotv.com","tvazteca.com/aztecanoticias","infobae.com/mexico",
    "la-lista.com/mexico","oem.com.mx/la-prensa","oem.com.mx/elsoldemexico","proceso.com.mx","vertigopolitico.com","sdpnoticias.com",
    "lasillarota.com/nacion","excelsior.com.mx","letraslibres.com","elpais.com/mexico/","mvsnoticias.com","latimes.com","jornada.com.mx",
    "cnnespanol.cnn.com/","nytimes.com","mexico.quadratin.com.mx","milenio.com","informador.mx","washingtonpost.com",
    "realestatemarket.com.mx/mercado-inmobiliario-home/industrial","alfamexico.com/category/noticias-inmobiliarias/",
    "inmobiliare.com/","jll.com.mx/es/newsroom","eleconomista.com.mx/tags/sector-inmobiliario","milenio.com/temas/sector-inmobiliario",
    "somosindustria.com/", "mexicoindustry.com/","thelogisticsworld.com/","bloomberglinea.com/latinoamerica/mexico/"
]

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/109.0.0.0 Safari/537.36"
    )
}

# === FUNCIONES AUXILIARES ===

def normalizar(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.lower().strip()
    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join([c for c in texto if not unicodedata.combining(c)])
    return texto

def parsear_fecha_pub(pub_date_str):
    """
    Intenta parsear la fecha del feed RSS de Google News.
    Devuelve objeto datetime con timezone.
    """
    try:
        dt = parser.parse(pub_date_str)
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=timezone.utc)
        return dt
    except Exception:
        return None

# === SCRAPING DE NOTICIAS POR TÉRMINO Y MEDIO ===

noticias = []
ahora = datetime.now(timezone.utc)

for termino in terminos_busqueda:
    print(f"\n🔍 Buscando: {termino}")

    for medio in medios:
        dominio = urlparse("https://" + medio).netloc
        # Configuración para México
        url = (
            f"https://news.google.com/rss/search?"
            f"q={termino}+site:{dominio}"
            f"&hl=es-419&gl=MX&ceid=MX:es"
        )

        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"⚠️ Error al obtener noticias de {dominio}")
            continue

        soup = BeautifulSoup(response.content, "xml")
        items = soup.find_all("item")

        for item in items:
            title = item.title.text if item.title else ""
            link = item.link.text if item.link else ""
            pub_date_str = item.pubDate.text if item.pubDate else ""

            pub_date = parsear_fecha_pub(pub_date_str)
            if pub_date is None:
                continue

            # Filtrado por fecha
            if modo == "exacto":
                if pub_date.date() != fecha_objetivo:
                    continue
            elif modo == "rango":
                if ahora - pub_date > timedelta(days=dias_rango):
                    continue

                source = item.source.text if item.source else ""
                description = item.description.text if item.description else ""
                contenido = f"{title} {description}".lower()
                if termino.lower() in contenido:
                    if debug:
                        print(f"📰 {title} | {pub_date.date()} | {source}")
                    noticias.append([
                        pub_date.astimezone(timezone.utc),
                        title,
                        link,
                        termino,
                        source,  # aquí guardamos el medio
                    ])


# === BLOQUE EXTRA: TITULARES GENERALES PARA "AGENDA NACIONAL" ===

from urllib.parse import quote_plus  # ya lo tienes arriba, pero no pasa nada si está repetido

print("\n🔍 Buscando titulares generales de México (agenda_nacional)")

# Construimos una query OR con las palabras de agenda nacional
# Ejemplo: "Sheinbaum OR aranceles OR trump"
query_agenda = " OR ".join(termino_agenda_nacional)
query_agenda_encoded = quote_plus(query_agenda)

url_agenda = (
    "https://news.google.com/rss/search?"
    f"q={query_agenda_encoded}"
    "&hl=es-419&gl=MX&ceid=MX:es"
)

response = requests.get(url_agenda, headers=headers)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "xml")
    items = soup.find_all("item")

    # Tomamos solo 4–5 ítems (ajustable)
    max_agenda = 15
    count = 0

    for item in items:
        if count >= max_agenda:
            break

        title = item.title.text if item.title else ""
        link = item.link.text if item.link else ""
        pub_date_str = item.pubDate.text if item.pubDate else ""
        pub_date = parsear_fecha_pub(pub_date_str)
        if pub_date is None:
            continue

        # Mismo filtro de fecha que en el resto del scraper
        if modo == "exacto":
            if pub_date.date() != fecha_objetivo:
                continue
        elif modo == "rango":
            if ahora - pub_date > timedelta(days=dias_rango):
                continue

        if debug:
            print(f"📰 [AGENDA] {title} | {pub_date.date()}")

        # El campo 'medio' lo dejamos vacío; luego se llenará con el split " - "
        noticias.append([
            pub_date.astimezone(timezone.utc),
            title,
            link,
            "agenda_nacional",
            ""
        ])
        count += 1
else:
    print("⚠️ No se pudieron obtener titulares generales de México")


# === GUARDAR RESULTADOS EN CSV NOTICIAS_DAP ===

carpeta_salida = "."  # raíz del proyecto; cambia si quieres "noticias"
os.makedirs(carpeta_salida, exist_ok=True)

if noticias:
    df = pd.DataFrame(
        noticias,
        columns=["fecha_dt", "titular_raw", "enlace", "termino", "medio"]
    )
    # Separar "titulo - Medio"
    partes = df["titular_raw"].str.split(" - ", n=1, expand=True)
    
    # Título limpio (antes del " - ")
    df["titular"] = partes[0]
    
    # Medio de comunicación (después del " - ")
    df["medio"] = partes[1].fillna("")

    # Convertir a fecha (sin hora) y a string YYYY-MM-DD
    df["fecha_dt"] = pd.to_datetime(df["fecha_dt"], errors="coerce")
    df = df[df["fecha_dt"].notna()]
    df["fecha"] = df["fecha_dt"].dt.strftime("%Y-%m-%d")
    # Nos quedamos solo con las columnas que quieres
    df_final = df[["fecha", "titular", "termino", "enlace", "medio"]].drop_duplicates()

    # Nombre del archivo
    sufijo_fecha = (
        fecha_objetivo.strftime("%Y-%m-%d")
        if modo == "exacto"
        else ahora.strftime("%Y-%m-%d")
    )
    nombre_archivo = os.path.join(
        carpeta_salida,
        f"noticias_dap_{sufijo_fecha}.csv"
    )

    df_final.to_csv(nombre_archivo, index=False, encoding="utf-8-sig")
    print(f"\n✅ Archivo guardado como '{nombre_archivo}' con {len(df_final)} noticias")
else:
    print("\n❌ No se encontraron noticias que cumplan los filtros.")



🔍 Buscando: industria alimentaria
📰 ¿Cómo cambiarán los medicamentos para perder peso a la industria alimentaria? Aquí te contamos - Forbes México | 2026-02-01 | Forbes México

🔍 Buscando: cemento

🔍 Buscando: gas
📰 Cuba se queda sin gas LP: Jamaica se ‘rehúsa’ a venderle petróleo por ‘terror’ a aranceles de Trump - El Financiero | 2026-02-02 | El Financiero
📰 Venezuela exporta por primera vez gas licuado de petróleo, PDVSA marca un hito energético - El Universal | 2026-02-02 | El Universal
📰 Tres fugas de gas en cinco meses en Coacalco - El Universal | 2026-02-01 | El Universal
📰 Gas Natural y TMEC - El Heraldo de México | 2026-02-02 | El Heraldo de México
📰 Explota una casa en la colonia Axotla en Álvaro Obregón por acumulación de gas - El Heraldo de México | 2026-02-02 | El Heraldo de México
📰 Graban momento exacto en que ocurre explosión por acumulación de gas en colonia Axotla | VIDEO FUERTE - El Heraldo de México | 2026-02-03 | El Heraldo de México
📰 ¿Dónde cargar gasolina más b